### Inferring the Last Name

We use a similar strategy as in last_sex to infer last names. We begin by 'exploding' the name by space. We then:

1. We remove records of people born before 1900 as we don't think those records are reliable
2. Remove all records where name is just one word as we cannot separate last name from first name
3. We remove all records where last name is < 2 char 
4. We remove last names with non-alphabetical characters
5. Lastly, we filter on last names that are shared by at least a 1000 people. 

Given we only have data on head of households and given a very large majority of HoH are men, we cannot use the sex ratio test to separate first names from last names

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/opt/data/secc/v2/secc_state_ln_sex_social_cat_birthyear_recode.csv.gz')
df

,state,gender,social_cat,birth_year,last_name
0,0,m,t,1979,mai
1,0,m,t,1986,tega
2,0,m,t,1954,tega
3,0,m,t,1964,tega
4,0,m,t,1984,mai
...,...,...,...,...,...
140194816,18,m,c,1976,maity
140194817,18,m,c,1974,maity
140194818,18,m,c,1969,maity
140194819,18,m,o,1949,mahapatra


In [3]:
df.dropna(subset=['last_name'], inplace=True)
df

,state,gender,social_cat,birth_year,last_name
0,0,m,t,1979,mai
1,0,m,t,1986,tega
2,0,m,t,1954,tega
3,0,m,t,1964,tega
4,0,m,t,1984,mai
...,...,...,...,...,...
140194816,18,m,c,1976,maity
140194817,18,m,c,1974,maity
140194818,18,m,c,1969,maity
140194819,18,m,o,1949,mahapatra


In [4]:
df.drop(df[~df.last_name.str.isalpha()].index, inplace=True)
df

,state,gender,social_cat,birth_year,last_name
0,0,m,t,1979,mai
1,0,m,t,1986,tega
2,0,m,t,1954,tega
3,0,m,t,1964,tega
4,0,m,t,1984,mai
...,...,...,...,...,...
140194816,18,m,c,1976,maity
140194817,18,m,c,1974,maity
140194818,18,m,c,1969,maity
140194819,18,m,o,1949,mahapatra


In [5]:
df.drop(df[df.last_name.str.len() < 2].index, inplace=True)
df

,state,gender,social_cat,birth_year,last_name
0,0,m,t,1979,mai
1,0,m,t,1986,tega
2,0,m,t,1954,tega
3,0,m,t,1964,tega
4,0,m,t,1984,mai
...,...,...,...,...,...
140194816,18,m,c,1976,maity
140194817,18,m,c,1974,maity
140194818,18,m,c,1969,maity
140194819,18,m,o,1949,mahapatra


In [6]:
df.drop(df[df.birth_year < 1900].index, inplace=True)
df

,state,gender,social_cat,birth_year,last_name
0,0,m,t,1979,mai
1,0,m,t,1986,tega
2,0,m,t,1954,tega
3,0,m,t,1964,tega
4,0,m,t,1984,mai
...,...,...,...,...,...
140194816,18,m,c,1976,maity
140194817,18,m,c,1974,maity
140194818,18,m,c,1969,maity
140194819,18,m,o,1949,mahapatra


In [7]:
%%time
adf = df[df['last_name'].groupby(df['last_name']).transform('size')>1000]
adf

CPU times: user 28.8 s, sys: 9.44 s, total: 38.2 s
Wall time: 42 s


,state,gender,social_cat,birth_year,last_name
640,0,m,t,1960,kri
641,0,m,t,1955,kri
642,0,m,t,1961,kri
643,0,m,t,1940,kri
644,0,m,t,1940,kri
...,...,...,...,...,...
140194816,18,m,c,1976,maity
140194817,18,m,c,1974,maity
140194818,18,m,c,1969,maity
140194819,18,m,o,1949,mahapatra


In [8]:
adf['last_name'].groupby(adf['last_name']).size()

last_name
aade         6293
aadivasi     6992
aadiwasi     6707
aaher        1570
aahir        1904
            ...  
zala        35231
zalte        1215
zapadiya     1393
zende        1661
zore         1983
Name: last_name, Length: 6661, dtype: int64

In [9]:
gdf = adf.groupby(['state', 'birth_year', 'last_name']).agg({'gender': 'value_counts', 'social_cat': 'value_counts'}).unstack()

In [10]:
gdf

gender                   social_cat                  
                      c    f    m   o   t          c   f   m    o    t
0  1901 tamang      NaN  NaN  2.0 NaN NaN        NaN NaN NaN  2.0  NaN
   1903 chakma      NaN  NaN  2.0 NaN NaN        NaN NaN NaN  2.0  NaN
   1904 ete         NaN  NaN  2.0 NaN NaN        NaN NaN NaN  NaN  2.0
   1905 bahadur     NaN  NaN  2.0 NaN NaN        NaN NaN NaN  2.0  NaN
        deori       NaN  NaN  2.0 NaN NaN        NaN NaN NaN  NaN  2.0
...                 ...  ...  ...  ..  ..        ...  ..  ..  ...  ...
18 2011 tanti       NaN  2.0  NaN NaN NaN        NaN NaN NaN  2.0  NaN
        tarafdar    NaN  NaN  2.0 NaN NaN        2.0 NaN NaN  NaN  NaN
        topno       NaN  NaN  1.0 NaN NaN        NaN NaN NaN  1.0  NaN
        tudu        NaN  1.0  NaN NaN NaN        NaN NaN NaN  NaN  1.0
        xalxo       NaN  NaN  1.0 NaN NaN        NaN NaN NaN  1.0  NaN

[1356699 rows x 10 columns]

In [11]:
gdf.reset_index(inplace=True)
gdf

level_0 level_1   level_2 gender                   social_cat          \
                                       c    f    m   o   t          c   f   m   
0             0    1901    tamang    NaN  NaN  2.0 NaN NaN        NaN NaN NaN   
1             0    1903    chakma    NaN  NaN  2.0 NaN NaN        NaN NaN NaN   
2             0    1904       ete    NaN  NaN  2.0 NaN NaN        NaN NaN NaN   
3             0    1905   bahadur    NaN  NaN  2.0 NaN NaN        NaN NaN NaN   
4             0    1905     deori    NaN  NaN  2.0 NaN NaN        NaN NaN NaN   
...         ...     ...       ...    ...  ...  ...  ..  ..        ...  ..  ..   
1356694      18    2011     tanti    NaN  2.0  NaN NaN NaN        NaN NaN NaN   
1356695      18    2011  tarafdar    NaN  NaN  2.0 NaN NaN        2.0 NaN NaN   
1356696      18    2011     topno    NaN  NaN  1.0 NaN NaN        NaN NaN NaN   
1356697      18    2011      tudu    NaN  1.0  NaN NaN NaN        NaN NaN NaN   
1356698      18    2011     xalxo    NaN  NaN  1.0 NaN NaN        NaN NaN NaN   

                   
           o    t  
0        2.0  NaN  
1        2.0  NaN  
2        NaN  2.0  
3        2.0  NaN  
4        NaN  2.0  
...      ...  ...  
1356694  2.0  NaN  
1356695  NaN  NaN  
1356696  1.0  NaN  
1356697  NaN  1.0  
1356698  1.0  NaN  

[1356699 rows x 13 columns]

In [12]:
gdf.columns = ['_'.join(col).strip() for col in gdf.columns.values]
gdf

,level_0_,level_1_,level_2_,gender_c,gender_f,gender_m,gender_o,gender_t,social_cat_c,social_cat_f,social_cat_m,social_cat_o,social_cat_t
0,0,1901,tamang,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1,0,1903,chakma,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
2,0,1904,ete,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,0,1905,bahadur,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,0,1905,deori,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356694,18,2011,tanti,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1356695,18,2011,tarafdar,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN
1356696,18,2011,topno,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1356697,18,2011,tudu,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [13]:
gdf = gdf[['level_0_', 'level_1_', 'level_2_', 'gender_m', 'gender_f', 'social_cat_c', 'social_cat_o', 'social_cat_t']]
gdf.columns = ['state', 'birth_year', 'last_name', 'n_male', 'n_female', 'n_sc', 'n_other', 'n_st']
gdf

,state,birth_year,last_name,n_male,n_female,n_sc,n_other,n_st
0,0,1901,tamang,2.0,NaN,NaN,2.0,NaN
1,0,1903,chakma,2.0,NaN,NaN,2.0,NaN
2,0,1904,ete,2.0,NaN,NaN,NaN,2.0
3,0,1905,bahadur,2.0,NaN,NaN,2.0,NaN
4,0,1905,deori,2.0,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
1356694,18,2011,tanti,NaN,2.0,NaN,2.0,NaN
1356695,18,2011,tarafdar,2.0,NaN,2.0,NaN,NaN
1356696,18,2011,topno,1.0,NaN,NaN,1.0,NaN
1356697,18,2011,tudu,NaN,1.0,NaN,NaN,1.0


In [14]:
gdf.fillna(0, inplace=True)
gdf.astype({'n_male': int, 'n_female': int, 'n_sc': int, 'n_st': int, 'n_other': int})

/opt/venv37/lib/python3.7/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,state,birth_year,last_name,n_male,n_female,n_sc,n_other,n_st
0,0,1901,tamang,2,0,0,2,0
1,0,1903,chakma,2,0,0,2,0
2,0,1904,ete,2,0,0,0,2
3,0,1905,bahadur,2,0,0,2,0
4,0,1905,deori,2,0,0,0,2
...,...,...,...,...,...,...,...,...
1356694,18,2011,tanti,0,2,0,2,0
1356695,18,2011,tarafdar,2,0,2,0,0
1356696,18,2011,topno,1,0,0,1,0
1356697,18,2011,tudu,0,1,0,0,1


In [15]:
sdf = pd.read_csv('secc_state.csv')
sdf

,state,code
0,arunachal pradesh,0
1,assam,1
2,bihar,2
3,chhattisgarh,3
4,gujarat,4
5,haryana,5
6,kerala,6
7,madhya pradesh,7
8,maharashtra,8
9,mizoram,9


In [16]:
odf = gdf.rename(columns={'state': 'code'}).merge(sdf)
del odf['code']
odf

,birth_year,last_name,n_male,n_female,n_sc,n_other,n_st,state
0,1901,tamang,2.0,0.0,0.0,2.0,0.0,arunachal pradesh
1,1903,chakma,2.0,0.0,0.0,2.0,0.0,arunachal pradesh
2,1904,ete,2.0,0.0,0.0,0.0,2.0,arunachal pradesh
3,1905,bahadur,2.0,0.0,0.0,2.0,0.0,arunachal pradesh
4,1905,deori,2.0,0.0,0.0,0.0,2.0,arunachal pradesh
...,...,...,...,...,...,...,...,...
1356694,2011,tanti,0.0,2.0,0.0,2.0,0.0,west bengal
1356695,2011,tarafdar,2.0,0.0,2.0,0.0,0.0,west bengal
1356696,2011,topno,1.0,0.0,0.0,1.0,0.0,west bengal
1356697,2011,tudu,0.0,1.0,0.0,0.0,1.0,west bengal


### Fix columns datatype

In [ ]:
odf = odf.astype({'n_male': int, 'n_female': int, 'n_sc': int, 'n_st': int, 'n_other': int})
odf

In [17]:
odf.to_csv('/opt/data/secc/v2/secc_all_state_year_ln_outkast.csv.gz', index=False, compression='gzip')

In [18]:
bdf = odf.groupby(['last_name']).agg({'n_sc': 'sum', 'n_other': 'sum', 'n_st': 'sum', 'n_male': 'sum', 'n_female': 'sum'}).reset_index()
bdf

,last_name,n_sc,n_other,n_st,n_male,n_female
0,aade,19.0,5663.0,611.0,5766.0,527.0
1,aadivasi,232.0,531.0,6229.0,6493.0,499.0
2,aadiwasi,84.0,874.0,5749.0,6121.0,586.0
3,aaher,77.0,1346.0,147.0,1422.0,148.0
4,aahir,67.0,1555.0,282.0,1711.0,193.0
...,...,...,...,...,...,...
6656,zala,667.0,34550.0,14.0,31966.0,3265.0
6657,zalte,310.0,584.0,321.0,1076.0,139.0
6658,zapadiya,41.0,1345.0,7.0,1328.0,65.0
6659,zende,582.0,1078.0,1.0,1435.0,226.0
